# Credit Score Binning

## Sources
- Dataset is "All Lending Club loan data: 2007 through current Lending Club accepted and rejected loan data" from Kaggle.

## Imports

In [200]:
# for preprocessing
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

# for visualizations
import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

# for modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold

# for evaluation
from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import time

# for suppressing system warnings
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = RuntimeWarning)

In [120]:
df_acc = pd.read_csv('accepted_2007_to_2018Q4.csv')
df_rej = pd.read_csv('rejected_2007_to_2018Q4.csv')

/Users/macuser/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [121]:
original_df_acc = df_acc.copy()
original_df_rej = df_rej.copy()

## Pipeline

In [122]:
class bold:
    start = '\033[1m'
    end   = '\033[0m'

def map_loan_status(loan_status): 
    if loan_status == 'Fully Paid':
        return 0
    else:
        return 1
    
def information(df, target = 'target'):
    # Prints typically useful statistical information about given dataframe.
    print("This dataframe consists of ", df.shape[1], " columns and", df.shape[0], " rows.")
    print("This dataframe consists of ", df.isnull().sum().sum(), " null entires.")
    print("This dataframe consists of ", df[df.duplicated()].shape[0], " duplicate rows.")
    print(df[df[target] == 1].shape[0], " rows belong to class target = 1.")
    print(df[df[target] == 0].shape[0], " rows belong to class target = 0.")
    print("")
    print(bold.start, "Notable statistics of numeric features in this dataset:", bold.end)
    print("")
    print(df.describe())
    print("")
    print(bold.start, "Object type of features in this dataset:", bold.end)
    print("")
    df.info()
    
def dist_box(x, title = ''):
    fig, axes = plt.subplots(nrows = 2, ncols = 1, figsize = (14.4, 7.2))
    for ax in axes:
        sns.kdeplot(x, shade = False, ax = ax)
        kdeline = ax.lines[0]
        xs = kdeline.get_xdata()
        ys = kdeline.get_ydata()
        if ax == axes[0]:
            middle = x.mean()
            sdev = x.std()
            left = middle - sdev
            right = middle + sdev
            ax.set_title('Mean and SD')
        else:
            left, middle, right = np.percentile(x, [25, 50, 75])
            ax.set_title('Median and Quartiles')
        ax.vlines(middle, 0, np.interp(middle, xs, ys), ls = ':')
        ax.fill_between(xs, 0, ys, alpha = 0.2)
        ax.fill_between(xs, 0, ys, where = (left <= xs) & (xs <= right), interpolate = True, alpha = 0.2)
    fig.suptitle(title, fontsize = 16)
    plt.show()
    
def logit_objective(space):
    clf = LogisticRegression(**params, random_state = 60, verbose = True)
    cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 60)
    score = cross_val_score(clf, X_train, y_train, cv = cv, scoring = 'f1_macro')

    best_score = max(score)

    loss = 1 - best_score

    return {'loss': loss, 'params': params, 'status': STATUS_OK}

def xgb_objective(space):
    clf = xgb.XGBClassifier(
        learning_rate = space['learning_rate'],
        max_depth = int(space['max_depth']),
        n_estimators = space['n_estimators'],
        objective = space['objective'],
        gamma = space['gamma'],
        reg_alpha = int(space['reg_alpha']),
        min_child_weight = int(space['min_child_weight']),
    )
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set = evaluation, eval_metric = "auc",
            early_stopping_rounds = 10, verbose = True)    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred > 0.5)
    return {'loss': -accuracy, 'status': STATUS_OK }

def f_score(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1 - f1_score(y_true, np.round(y_pred))
    return 'f1_err', err
    
def evaluate(ytest, y_pred, speed):
    confusion = confusion_matrix(y_test, y_pred)
    fbeta = fbeta_score(y_test, y_pred, average = 'binary', beta = 1.2)
    print(bold.start, "Classification Report:", bold.end)
    print("")
    print(classification_report(y_test, y_pred))
    print(bold.start, "F - 1.2 Score:", bold.end)
    print("")
    print("{:.6f}".format(fbeta))
    print("")
    print(bold.start, "Prediction speed:", bold.end)
    print("")
    print("{:.6f} seconds".format(speed))
    
def roc(model_string, y_test, y_pred):
    roc_auc = roc_auc_score(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    plt.figure(figsize = (9.6, 7.2))
    plt.grid()
    plt.plot(fpr, tpr, label = model_string + ' (AUC = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc = "best")
    plt.show()

## EDA

In [123]:
df_acc.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
5,68426831,NaN,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
6,68476668,NaN,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
7,67275481,NaN,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
8,68466926,NaN,10000.0,10000.0,10000.0,36 months,6.49,306.45,A,A2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
9,68616873,NaN,8000.0,8000.0,8000.0,36 months,11.48,263.74,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


### Mapping Target Variable
- In the following cell, take a look at our target variable 'loan_status'.

In [124]:
df_acc['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

- Filter out rows whose 'loan_status' value isn't one of 'Fully Paid', 'Charged Off', or 'Default'.

In [125]:
loan_status_filter = df_acc['loan_status'].isin(['Fully Paid','Charged Off','Default'])
df_acc = df_acc[loan_status_filter]
df_acc['loan_status'].value_counts()

Fully Paid     1076751
Charged Off     268559
Default             40
Name: loan_status, dtype: int64

- Create a new feature 'late_loan' whose values are obtained by mapping loan_status to space [0, 1]. 'Fully Paid' gets mapped to 0, and 'Charged Off' gets mapped to 1.

In [126]:
df_acc['late_loan'] = df_acc['loan_status'].map(map_loan_status)
df_acc = df_acc.drop('loan_status', axis = 1)
df_acc['late_loan'].value_counts()

0    1076751
1     268599
Name: late_loan, dtype: int64

In [127]:
information(df_acc, target = 'late_loan')

This dataframe consists of  151  columns and 1345350  rows.
This dataframe consists of  70409686  null entires.
This dataframe consists of  0  duplicate rows.
268599  rows belong to class target = 1.
1076751  rows belong to class target = 0.

 Notable statistics of numeric features in this dataset: 

       member_id     loan_amnt   funded_amnt  funded_amnt_inv      int_rate  \
count        0.0  1.345350e+06  1.345350e+06     1.345350e+06  1.345350e+06   
mean         NaN  1.441997e+04  1.441155e+04     1.438914e+04  1.323971e+01   
std          NaN  8.717098e+03  8.713166e+03     8.715542e+03  4.768782e+00   
min          NaN  5.000000e+02  5.000000e+02     0.000000e+00  5.310000e+00   
25%          NaN  8.000000e+03  8.000000e+03     7.875000e+03  9.750000e+00   
50%          NaN  1.200000e+04  1.200000e+04     1.200000e+04  1.274000e+01   
75%          NaN  2.000000e+04  2.000000e+04     2.000000e+04  1.599000e+01   
max          NaN  4.000000e+04  4.000000e+04     4.000000e+04  3.0

Observations:
- There are about 2.3 millions of missing values in the dataframe.
- The data has imbalanced classes.
- There are 151 features, each with more than 1.3 million entries.

### Dimensionality Reduction
- Drop features with more than 10% missing entries.

In [197]:
threshold = df_acc.shape[0] * (1 - 0.1)
df_acc = df_acc.dropna(thresh = threshold, axis = 'columns')
df_acc.shape[1], original_df_acc.shape[1]

(92, 151)

In [198]:
for i in range(len(df_acc.loc[1])):
    print(df_acc.columns[i], ": ", df_acc.loc[1][i])

id :  68355089
loan_amnt :  24700.0
funded_amnt :  24700.0
funded_amnt_inv :  24700.0
term :   36
int_rate :  11.99
installment :  820.28
grade :  C
sub_grade :  C1
emp_title :  Engineer
emp_length :  10+ years
home_ownership :  MORTGAGE
annual_inc :  65000.0
verification_status :  Not Verified
issue_d :  Dec-2015
pymnt_plan :  n
url :  https://lendingclub.com/browse/loanDetail.action?loan_id=68355089
purpose :  small_business
title :  Business
zip_code :  577xx
addr_state :  SD
dti :  16.06
delinq_2yrs :  1.0
earliest_cr_line :  Dec-1999
fico_range_low :  715.0
fico_range_high :  719.0
inq_last_6mths :  4.0
open_acc :  22.0
pub_rec :  0.0
revol_bal :  21470.0
revol_util :  19.2
total_acc :  38.0
initial_list_status :  w
out_prncp :  0.0
out_prncp_inv :  0.0
total_pymnt :  25679.66
total_pymnt_inv :  25679.66
total_rec_prncp :  24700.0
total_rec_int :  979.66
total_rec_late_fee :  0.0
recoveries :  0.0
collection_recovery_fee :  0.0
last_pymnt_d :  Jun-2016
last_pymnt_amnt :  926.35
la

In [199]:
df_acc1 = df_acc.drop(['id','url','zip_code','policy_code','application_type','last_pymnt_d','last_credit_pull_d','verification_status','pymnt_plan','funded_amnt','funded_amnt_inv','sub_grade','out_prncp','out_prncp_inv','total_pymnt_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','initial_list_status','earliest_cr_line'],axis =1)
df_acc1['emp_length'].fillna('Unknown',inplace = True)
df_acc1['emp_title'].fillna('Unknown',inplace = True)
df_acc1['title'].fillna('Unknown',inplace = True)
df_acc1['revol_util'].fillna(df_acc1['revol_util'].mean(),inplace = True)
df_acc1['collections_12_mths_ex_med'].fillna(df_acc1['collections_12_mths_ex_med'].mean(),inplace = True)
df_acc1.isnull().sum()

loan_amnt                         0
term                              0
int_rate                          0
installment                       0
grade                             0
                              ...  
total_il_high_credit_limit    67527
hardship_flag                     0
disbursement_method               0
debt_settlement_flag              0
late_loan                         0
Length: 68, dtype: int64

In [201]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(df_acc1, df_acc1['late_loan'])